In [29]:
%pip install pandas
%pip install plotly

import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, timedelta

# --- Define Chart Parameters ---
symbol = 'AAPL'
# Define the specific date for which you want to create the chart.
# Note: Finnhub free tier historical data might be limited, often 1-day delayed.
chart_date_str = '2023-10-12'

# --- Convert Date to Unix Timestamps ---
start_datetime = datetime.strptime(chart_date_str, '%Y-%m-%d')
start_timestamp = int(start_datetime.timestamp())

# Calculate the end timestamp for the *end of the day*.
end_datetime = start_datetime + timedelta(days=1) - timedelta(seconds=1)
end_timestamp = int(end_datetime.timestamp())

# --- Fetch Historical Data ---
print(f"Fetching {symbol} data for {chart_date_str}...")
print(f"Start Timestamp: {start_timestamp} (Unix)")
print(f"End Timestamp: {end_timestamp} (Unix)")

try:
    # Fetch historical data in 15-minute intervals.
    response = finnhub_client.stock_candles(symbol, '15', start_timestamp, end_timestamp)

    # Check for empty or erroneous response
    if not response or ('s' in response and response['s'] == 'no_data'):
        print(f"No data found for {symbol} on {chart_date_str}. Check symbol, date, or API key access.")
        print(f"Finnhub Response: {response}")
    elif 's' in response and response['s'] == 'ok':
        # --- Extract Data Attributes from the response ---
        timestamps = response['t']
        opens = response['o']
        highs = response['h']
        lows = response['l']
        closes = response['c']
        volumes = response['v']

        # --- Convert Timestamps to Datetime Objects ---
        datetime_index = pd.to_datetime(timestamps, unit='s')

        # --- Create a DataFrame from the retrieved data ---
        df = pd.DataFrame({
            'open': opens,
            'high': highs,
            'low': lows,
            'close': closes,
            'volume': volumes
        }, index=datetime_index)

        # Sort the DataFrame by timestamp to ensure proper chronological order for plotting
        df = df.sort_index()

        # --- Create a Candlestick Chart using Plotly ---
        fig = go.Figure(data=[go.Candlestick(x=df.index,
                                             open=df['open'],
                                             high=df['high'],
                                             low=df['low'],
                                             close=df['close'])])

        fig.update_layout(
            title=f'Candlestick Chart for {symbol} on {chart_date_str}',
            xaxis_title='Time',
            yaxis_title='Price',
            xaxis_rangeslider_visible=False,
            template='plotly_white'
        )

        # Show the plot
        fig.show()

    else:
        print(f"An unexpected response status received: {response.get('s', 'N/A')}")
        print(f"Full response: {response}")

except Exception as e:
    print(f"An error occurred: {e}")
    print("Please ensure your API key is correct and you have an active internet connection.")


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Fetching AAPL data for 2023-10-12...
Start Timestamp: 1697049000 (Unix)
End Timestamp: 1697135399 (Unix)
An error occurred: FinnhubAPIException(status_code: 403): You don't have access to this resource.
Please ensure your API key is correct and you have an active internet connection.


In [30]:
try:
	print(finnhub_client.quote('AAPL'))
except Exception as e:
	print(f"Error fetching AAPL quote: {e}")

try:
	print(finnhub_client.quote('AMD'))
except Exception as e:
	print(f"Error fetching AMD quote: {e}")

# General news
try:
	print(finnhub_client.general_news('forex', min_id=0))
except Exception as e:
	print(f"Error fetching general news: {e}")

try:
	print(finnhub_client.technical_indicator(symbol="AAPL", resolution='D', _from=1583098857, to=1584308457, indicator='rsi', indicator_fields={"timeperiod": 3}))
except Exception as e:
	print(f"Error fetching technical indicator: {e}")

{'c': 201, 'd': 4.42, 'dp': 2.2484, 'h': 201.7, 'l': 196.855, 'o': 198.235, 'pc': 196.58, 't': 1750536000}
{'c': 128.24, 'd': 1.45, 'dp': 1.1436, 'h': 132.805, 'l': 127.55, 'o': 129, 'pc': 126.79, 't': 1750536000}
[{'category': 'forex', 'datetime': 1750550201, 'headline': 'BREAKING: Trump Claims U.S. Successfully Struck Iran Nuclear Sites', 'id': 7491538, 'image': 'https://images.forexlive.com/fl-static/fl-thumbnail.jpg', 'related': '', 'source': 'Forexlive', 'summary': '<p>Former U.S. President Donald Trump announced on social media that the United States has completed what he called a "very successful attack" on three nuclear sites in Iran — Fordow, Natanz, and Esfahan. According to Trump’s post, a full payload of bombs was dropped on the primary target, Fordow, and all aircraft involved have exited Iranian airspace safely.</p><p>Trump praised the operation and the American military, declaring, “There is not another military in the World that could have done this.” He ended the state

In [31]:

try:
	result = finnhub_client.technical_indicator(
		symbol=symbol,
		resolution='D',
		_from=start_timestamp,
		to=end_timestamp,
		indicator='rsi',
		indicator_fields={"timeperiod": 3}
	)
	print(result)
except Exception as e:
	print(f"Error fetching technical indicator: {e}")
	print("You may not have access to this resource with your current API key.")

Error fetching technical indicator: FinnhubAPIException(status_code: 403): You don't have access to this resource.
You may not have access to this resource with your current API key.


In [32]:
# General news
print(finnhub_client.general_news('forex', min_id=0))


[{'category': 'forex', 'datetime': 1750550201, 'headline': 'BREAKING: Trump Claims U.S. Successfully Struck Iran Nuclear Sites', 'id': 7491538, 'image': 'https://images.forexlive.com/fl-static/fl-thumbnail.jpg', 'related': '', 'source': 'Forexlive', 'summary': '<p>Former U.S. President Donald Trump announced on social media that the United States has completed what he called a "very successful attack" on three nuclear sites in Iran — Fordow, Natanz, and Esfahan. According to Trump’s post, a full payload of bombs was dropped on the primary target, Fordow, and all aircraft involved have exited Iranian airspace safely.</p><p>Trump praised the operation and the American military, declaring, “There is not another military in the World that could have done this.” He ended the statement by calling for peace: “NOW IS THE TIME FOR PEACE!”</p><p>⚠️ Note: This claim has not yet been independently verified by the U.S. Department of Defense or international sources. The situation is developing.</p>

In [33]:
# Symbol lookup
print(finnhub_client.symbol_lookup('apple'))

{'count': 11, 'result': [{'description': 'Apple Inc', 'displaySymbol': 'AAPL', 'symbol': 'AAPL', 'type': 'Common Stock'}, {'description': 'Apple Hospitality REIT Inc', 'displaySymbol': 'APLE', 'symbol': 'APLE', 'type': 'Common Stock'}, {'description': 'Apple iSports Group Inc', 'displaySymbol': 'AAPI', 'symbol': 'AAPI', 'type': 'Common Stock'}, {'description': 'Apple Flavor & Fragrance Group Co Ltd', 'displaySymbol': '603020.SS', 'symbol': '603020.SS', 'type': 'Common Stock'}, {'description': 'Maui Land & Pineapple Company Inc', 'displaySymbol': 'MLP', 'symbol': 'MLP', 'type': 'Common Stock'}, {'description': 'Apple International Co Ltd', 'displaySymbol': '2788.T', 'symbol': '2788.T', 'type': 'Common Stock'}, {'description': 'Applepark Co Ltd', 'displaySymbol': '164A.T', 'symbol': '164A.T', 'type': 'Common Stock'}, {'description': 'Pineapple Resources Bhd', 'displaySymbol': 'PINEAPP.KL', 'symbol': 'PINEAPP.KL', 'type': 'Common Stock'}, {'description': 'Pineapple Inc', 'displaySymbol': 

In [34]:
# FDA Calendar
print(finnhub_client.fda_calendar())


[{'fromDate': '2025-07-18 09:00:00', 'toDate': '2025-07-18 16:00:00', 'eventDescription': 'July 18, 2025: Meeting of the Psychopharmacologic Drugs Advisory Committee Meeting Announcement - 07/18/2025', 'url': 'https://www.fda.gov/advisory-committees/advisory-committee-calendar/july-18-2025-meeting-psychopharmacologic-drugs-advisory-committee-meeting-announcement-07182025'}, {'fromDate': '2025-07-17 08:00:00', 'toDate': '2025-07-17 12:45:00', 'eventDescription': 'July 17, 2025: Meeting of the Oncologic Drugs Advisory Committee - 07/17/2025', 'url': 'https://www.fda.gov/advisory-committees/july-17-2025-meeting-oncologic-drugs-advisory-committee-07172025'}, {'fromDate': '2025-07-09 10:00:00', 'toDate': '2025-07-09 16:00:00', 'eventDescription': 'Pediatric Advisory Committee Meeting Announcement - 07/09/2025', 'url': 'https://www.fda.gov/advisory-committees/advisory-committee-calendar/pediatric-advisory-committee-meeting-announcement-07092025'}, {'fromDate': '2025-05-22 08:30:00', 'toDate'

In [35]:
%pip install finnhub-python

import finnhub

# Setup Finnhub client
finnhub_client = finnhub.Client(api_key="cen5uf2ad3i22rjjmfg0cen5uf2ad3i22rjjmfgg")

print(finnhub_client.earnings_calendar(_from="2023-01-10", to="2023-10-20", symbol="AAPL", international=False))

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
{'earningsCalendar': []}


In [36]:
import datetime

# Unix timestamps
timestamp1 = 1691918230
timestamp2 = 1691852249

# Convert to datetime objects
date1 = datetime.datetime.fromtimestamp(timestamp1)
date2 = datetime.datetime.fromtimestamp(timestamp2)

# Print the results
print("Timestamp 1:", date1)
print("Timestamp 2:", date2)


Timestamp 1: 2023-08-13 14:47:10
Timestamp 2: 2023-08-12 20:27:29


In [37]:
# Fetch one-minute candlestick data for AAPL using valid timestamps and handle exceptions
try:
	res = finnhub_client.stock_candles(symbol, '1', start_timestamp, end_timestamp)
	print(res)
except Exception as e:
	print(f"Error fetching candlestick data: {e}")
	print("You may not have access to this resource with your current API key or the time range may be invalid.")


Error fetching candlestick data: FinnhubAPIException(status_code: 403): You don't have access to this resource.
You may not have access to this resource with your current API key or the time range may be invalid.


In [38]:

# Basic financials
print(finnhub_client.company_basic_financials('AAPL', 'all'))

{'metric': {'10DayAverageTradingVolume': 55.45698, '13WeekPriceReturnDaily': -6.6159, '26WeekPriceReturnDaily': -18.4551, '3MonthADReturnStd': 49.613785, '3MonthAverageTradingVolume': 58.3943, '52WeekHigh': 260.1, '52WeekHighDate': '2024-12-26', '52WeekLow': 169.2101, '52WeekLowDate': '2025-04-08', '52WeekPriceReturnDaily': -6.2019, '5DayPriceReturnDaily': 2.3161, 'assetTurnoverAnnual': 1.0714, 'assetTurnoverTTM': 1.1673, 'beta': 1.1801103, 'bookValuePerShareAnnual': 3.7673, 'bookValuePerShareQuarterly': 4.4712, 'bookValueShareGrowth5Y': -5.85, 'capexCagr5Y': -2.08, 'cashFlowPerShareAnnual': 7.1978, 'cashFlowPerShareQuarterly': 6.5924, 'cashFlowPerShareTTM': 6.86253, 'cashPerSharePerShareAnnual': 4.3112, 'cashPerSharePerShareQuarterly': 3.2463, 'currentDividendYieldTTM': 0.5101, 'currentEv/freeCashFlowAnnual': 28.2346, 'currentEv/freeCashFlowTTM': 31.1935, 'currentRatioAnnual': 0.8673, 'currentRatioQuarterly': 0.8209, 'dividendGrowthRate5Y': 5.3, 'dividendPerShareAnnual': 0.9935, 'divi

In [39]:

# Earnings surprises
print(finnhub_client.company_earnings('TSLA', limit=5))

[{'actual': 0.27, 'estimate': 0.399, 'period': '2025-03-31', 'quarter': 1, 'surprise': -0.129, 'surprisePercent': -32.3308, 'symbol': 'TSLA', 'year': 2025}, {'actual': 0.73, 'estimate': 0.7777, 'period': '2024-12-31', 'quarter': 4, 'surprise': -0.0477, 'surprisePercent': -6.1335, 'symbol': 'TSLA', 'year': 2024}, {'actual': 0.72, 'estimate': 0.5959, 'period': '2024-09-30', 'quarter': 3, 'surprise': 0.1241, 'surprisePercent': 20.8256, 'symbol': 'TSLA', 'year': 2024}, {'actual': 0.52, 'estimate': 0.6361, 'period': '2024-06-30', 'quarter': 2, 'surprise': -0.1161, 'surprisePercent': -18.2518, 'symbol': 'TSLA', 'year': 2024}]


In [40]:

# Company News
# Need to use _from instead of from to avoid conflict
print(finnhub_client.company_news('AAPL', _from="2018-06-01", to="2020-06-10"))

[]


In [41]:

# Company Peers
print(finnhub_client.company_peers('AAPL'))

['AAPL', 'DELL', 'SMCI', '1337.HK', 'HPE', 'HPQ', 'WDC', 'NTAP', 'PSTG', 'IONQ']


In [42]:

# Company Profile 2
print(finnhub_client.company_profile2(symbol='AAPL'))

{'country': 'US', 'currency': 'USD', 'estimateCurrency': 'USD', 'exchange': 'NASDAQ NMS - GLOBAL MARKET', 'finnhubIndustry': 'Technology', 'ipo': '1980-12-12', 'logo': 'https://static2.finnhub.io/file/publicdatany/finnhubimage/stock_logo/AAPL.png', 'marketCapitalization': 3002101.0885106116, 'name': 'Apple Inc', 'phone': '14089961010', 'shareOutstanding': 14935.83, 'ticker': 'AAPL', 'weburl': 'https://www.apple.com/'}


In [43]:

# List country
print(finnhub_client.country())

[{'code2': 'NR', 'code3': 'NRU', 'codeNo': '520', 'country': 'Nauru', 'countryRiskPremium': None, 'currency': 'Australian Dollars', 'currencyCode': 'AUD', 'defaultSpread': None, 'equityRiskPremium': None, 'rating': None, 'region': 'Oceania', 'subRegion': 'Micronesia'}, {'code2': 'MF', 'code3': 'MAF', 'codeNo': '663', 'country': 'Saint Martin', 'countryRiskPremium': None, 'currency': 'Netherlands Antillean guilder', 'currencyCode': 'ANG', 'defaultSpread': None, 'equityRiskPremium': None, 'rating': None, 'region': 'Americas', 'subRegion': 'Latin America and the Caribbean'}, {'code2': 'GE', 'code3': 'GEO', 'codeNo': '268', 'country': 'Georgia', 'countryRiskPremium': 4.4, 'currency': 'Lari', 'currencyCode': 'GEL', 'defaultSpread': None, 'equityRiskPremium': 9, 'rating': 'Ba2', 'region': 'Asia', 'subRegion': 'Western Asia'}, {'code2': 'AQ', 'code3': 'ATA', 'codeNo': '10', 'country': 'Antarctica', 'countryRiskPremium': None, 'currency': 'Antarctican dollar', 'currencyCode': 'AQD', 'defaultSp

In [44]:

# Filings
print(finnhub_client.filings(symbol='AAPL', _from="2020-01-01", to="2020-06-11"))

[{'accessNumber': '0000320193-20-000056', 'symbol': 'AAPL', 'cik': '320193', 'form': '4', 'filedDate': '2020-05-12 00:00:00', 'acceptedDate': '2020-05-12 18:30:32', 'reportUrl': 'https://www.sec.gov/Archives/edgar/data/320193/000032019320000056/wf-form4_158932261319105.xml', 'filingUrl': 'https://www.sec.gov/Archives/edgar/data/320193/000032019320000056/0000320193-20-000056-index.html'}, {'accessNumber': '0001193125-20-139112', 'symbol': 'AAPL', 'cik': '320193', 'form': '8-K', 'filedDate': '2020-05-11 00:00:00', 'acceptedDate': '2020-05-11 16:31:15', 'reportUrl': 'https://www.sec.gov/Archives/edgar/data/320193/000119312520139112/d926511d8k.htm', 'filingUrl': 'https://www.sec.gov/Archives/edgar/data/320193/000119312520139112/0001193125-20-139112-index.html'}, {'accessNumber': '0001193125-20-133777', 'symbol': 'AAPL', 'cik': '320193', 'form': '424B2', 'filedDate': '2020-05-05 00:00:00', 'acceptedDate': '2020-05-05 16:31:51', 'reportUrl': 'https://www.sec.gov/Archives/edgar/data/320193/00

In [45]:

# Financials as reported
print(finnhub_client.financials_reported(symbol='AAPL', freq='annual'))

{'cik': '320193', 'data': [{'accessNumber': '0000320193-24-000123', 'symbol': 'AAPL', 'cik': '320193', 'year': 2024, 'quarter': 0, 'form': '10-K', 'startDate': '2023-10-01 00:00:00', 'endDate': '2024-09-28 00:00:00', 'filedDate': '2024-11-01 00:00:00', 'acceptedDate': '2024-11-01 06:01:36', 'report': {'bs': [{'concept': 'us-gaap_CashAndCashEquivalentsAtCarryingValue', 'unit': 'usd', 'label': 'Cash and cash equivalents', 'value': 29943000000.0}, {'concept': 'us-gaap_MarketableSecuritiesCurrent', 'unit': 'usd', 'label': 'Marketable securities', 'value': 35228000000.0}, {'concept': 'us-gaap_AccountsReceivableNetCurrent', 'unit': 'usd', 'label': 'Accounts receivable, net', 'value': 33410000000}, {'concept': 'us-gaap_NontradeReceivablesCurrent', 'unit': 'usd', 'label': 'Vendor non-trade receivables', 'value': 32833000000}, {'concept': 'us-gaap_InventoryNet', 'unit': 'usd', 'label': 'Inventories', 'value': 7286000000}, {'concept': 'us-gaap_OtherAssetsCurrent', 'unit': 'usd', 'label': 'Other 

In [46]:

# Forex exchanges
print(finnhub_client.forex_exchanges())

['oanda', 'fxcm', 'forex.com', 'fhfx', 'capital', 'pepperstone', 'fxpig', 'ic markets', 'pepperstoneuk', 'icmtrader']


In [47]:

# General news
print(finnhub_client.general_news('forex', min_id=0))

[{'category': 'forex', 'datetime': 1750550201, 'headline': 'BREAKING: Trump Claims U.S. Successfully Struck Iran Nuclear Sites', 'id': 7491538, 'image': 'https://images.forexlive.com/fl-static/fl-thumbnail.jpg', 'related': '', 'source': 'Forexlive', 'summary': '<p>Former U.S. President Donald Trump announced on social media that the United States has completed what he called a "very successful attack" on three nuclear sites in Iran — Fordow, Natanz, and Esfahan. According to Trump’s post, a full payload of bombs was dropped on the primary target, Fordow, and all aircraft involved have exited Iranian airspace safely.</p><p>Trump praised the operation and the American military, declaring, “There is not another military in the World that could have done this.” He ended the statement by calling for peace: “NOW IS THE TIME FOR PEACE!”</p><p>⚠️ Note: This claim has not yet been independently verified by the U.S. Department of Defense or international sources. The situation is developing.</p>

In [48]:

# IPO calendar
print(finnhub_client.ipo_calendar(_from="2023-05-01", to="2023-12-01"))


{'ipoCalendar': [{'date': '2023-12-01', 'exchange': 'NASDAQ Capital', 'name': 'Garden Stage Ltd', 'numberOfShares': 2500000, 'price': '4.00', 'status': 'priced', 'symbol': 'GSIW', 'totalSharesValue': 10000000}, {'date': '2023-11-30', 'exchange': 'NASDAQ Capital', 'name': 'Fenbo Holdings Ltd', 'numberOfShares': 1000000, 'price': '5.00', 'status': 'priced', 'symbol': 'FEBO', 'totalSharesValue': 5000000}, {'date': '2023-11-29', 'exchange': None, 'name': 'Concord Health Ltd', 'numberOfShares': None, 'price': None, 'status': 'filed', 'symbol': 'HHGJ', 'totalSharesValue': 1000000}, {'date': '2023-11-28', 'exchange': None, 'name': 'EvoAir Holdings Inc.', 'numberOfShares': None, 'price': None, 'status': 'filed', 'symbol': '', 'totalSharesValue': 15000000}, {'date': '2023-11-22', 'exchange': None, 'name': 'Flexi Group Holdings Ltd', 'numberOfShares': None, 'price': None, 'status': 'filed', 'symbol': 'FLXG', 'totalSharesValue': 17250000}, {'date': '2023-11-21', 'exchange': 'NYSE', 'name': 'COLOM

In [49]:

# Recommendation trends
print(finnhub_client.recommendation_trends('AAPL'))

[{'buy': 25, 'hold': 13, 'period': '2025-06-01', 'sell': 3, 'strongBuy': 15, 'strongSell': 1, 'symbol': 'AAPL'}, {'buy': 25, 'hold': 14, 'period': '2025-05-01', 'sell': 3, 'strongBuy': 14, 'strongSell': 0, 'symbol': 'AAPL'}, {'buy': 23, 'hold': 12, 'period': '2025-04-01', 'sell': 3, 'strongBuy': 12, 'strongSell': 1, 'symbol': 'AAPL'}, {'buy': 25, 'hold': 12, 'period': '2025-03-01', 'sell': 3, 'strongBuy': 12, 'strongSell': 1, 'symbol': 'AAPL'}]


In [50]:

# Stock symbols
print(finnhub_client.stock_symbols('US')[0:5])

[{'currency': 'USD', 'description': 'FT VEST US EQ MAX BUF DEC', 'displaySymbol': 'DECM', 'figi': 'BBG01R40MN45', 'isin': None, 'mic': 'BATS', 'shareClassFIGI': 'BBG01R40MNZ1', 'symbol': 'DECM', 'symbol2': '', 'type': 'ETP'}, {'currency': 'USD', 'description': 'QIAN YUAN BAIXING INC', 'displaySymbol': 'QYBX', 'figi': 'BBG000RQYRQ7', 'isin': None, 'mic': 'OOTC', 'shareClassFIGI': 'BBG001SV3KT9', 'symbol': 'QYBX', 'symbol2': '', 'type': 'Common Stock'}, {'currency': 'USD', 'description': 'PPL CORP', 'displaySymbol': 'PPL', 'figi': 'BBG000BRJL00', 'isin': None, 'mic': 'XNYS', 'shareClassFIGI': 'BBG001S5VC31', 'symbol': 'PPL', 'symbol2': '', 'type': 'Common Stock'}, {'currency': 'USD', 'description': 'OTTO ENERGY LTD', 'displaySymbol': 'OTTEF', 'figi': 'BBG000RL0WD8', 'isin': None, 'mic': 'OOTC', 'shareClassFIGI': 'BBG001SHCT84', 'symbol': 'OTTEF', 'symbol2': '', 'type': 'Common Stock'}, {'currency': 'USD', 'description': 'JANUS HENDERSON CORPORATE BO', 'displaySymbol': 'JLQD', 'figi': 'BB

In [51]:

# Covid-19
print(finnhub_client.covid19())


[{'state': 'California', 'case': 12711918, 'death': 112443, 'updated': '2024-11-18 18:02:03'}, {'state': 'Texas', 'case': 9190299, 'death': 104793, 'updated': '2024-11-18 18:02:03'}, {'state': 'Florida', 'case': 8048191, 'death': 95206, 'updated': '2024-11-18 18:02:03'}, {'state': 'New York', 'case': 7587861, 'death': 83374, 'updated': '2024-11-18 18:02:03'}, {'state': 'Illinois', 'case': 4136659, 'death': 42005, 'updated': '2024-11-18 18:02:03'}, {'state': 'Georgia', 'case': 3287483, 'death': 44069, 'updated': '2024-11-18 18:02:03'}, {'state': 'Pennsylvania', 'case': 3565499, 'death': 51480, 'updated': '2024-11-18 18:02:03'}, {'state': 'Ohio', 'case': 3741277, 'death': 43896, 'updated': '2024-11-18 18:02:03'}, {'state': 'North Carolina', 'case': 3501404, 'death': 29059, 'updated': '2024-11-18 18:02:03'}, {'state': 'Arizona', 'case': 2607545, 'death': 34402, 'updated': '2024-11-18 18:02:03'}, {'state': 'New Jersey', 'case': 3295020, 'death': 36873, 'updated': '2024-11-18 18:02:03'}, {'

In [52]:

# FDA Calendar
print(finnhub_client.fda_calendar())

[{'fromDate': '2025-07-18 09:00:00', 'toDate': '2025-07-18 16:00:00', 'eventDescription': 'July 18, 2025: Meeting of the Psychopharmacologic Drugs Advisory Committee Meeting Announcement - 07/18/2025', 'url': 'https://www.fda.gov/advisory-committees/advisory-committee-calendar/july-18-2025-meeting-psychopharmacologic-drugs-advisory-committee-meeting-announcement-07182025'}, {'fromDate': '2025-07-17 08:00:00', 'toDate': '2025-07-17 12:45:00', 'eventDescription': 'July 17, 2025: Meeting of the Oncologic Drugs Advisory Committee - 07/17/2025', 'url': 'https://www.fda.gov/advisory-committees/july-17-2025-meeting-oncologic-drugs-advisory-committee-07172025'}, {'fromDate': '2025-07-09 10:00:00', 'toDate': '2025-07-09 16:00:00', 'eventDescription': 'Pediatric Advisory Committee Meeting Announcement - 07/09/2025', 'url': 'https://www.fda.gov/advisory-committees/advisory-committee-calendar/pediatric-advisory-committee-meeting-announcement-07092025'}, {'fromDate': '2025-05-22 08:30:00', 'toDate'

In [53]:

# USA Spending
print(finnhub_client.stock_usa_spending("AAPL", "2021-01-01", "2022-06-15"))


{'data': [{'symbol': 'AAPL', 'recipientName': 'CLARIS INTERNATIONAL INC.', 'recipientParentName': 'APPLE INC', 'country': 'USA', 'totalValue': 3600, 'actionDate': '2022-04-25', 'performanceStartDate': '2022-04-25', 'performanceEndDate': '2023-05-10', 'awardingAgencyName': 'Smithsonian Institution', 'awardingSubAgencyName': 'Smithsonian Institution', 'awardingOfficeName': 'SMITHSONIAN TROPICAL RESEARCH INSTITUTE', 'performanceCountry': 'USA', 'performanceCity': 'SANTA CLARA', 'performanceCounty': 'SANTA CLARA', 'performanceState': '06', 'performanceZipCode': 'CALIFORNIA', 'performanceCongressionalDistrict': '950541164', 'awardDescription': 'FILEMAKER LICENSES FOR 10 WORKSTATIONS', 'naicsCode': '511210', 'permalink': 'https://www.usaspending.gov/award/CONT_AWD_33330522P00472238_3300_-NONE-_-NONE-/'}, {'symbol': 'AAPL', 'recipientName': 'APPLE INC', 'recipientParentName': 'APPLE INC', 'country': 'USA', 'totalValue': 17399.19, 'actionDate': '2022-04-21', 'performanceStartDate': '2015-08-18

In [54]:
# Lobbying
print(finnhub_client.stock_lobbying("AAPL", "2021-01-01", "2022-06-15"))


{'data': [{'symbol': 'AAPL', 'name': 'APPLE INC', 'description': '', 'country': 'US', 'year': 2020, 'period': 'Q4', 'documentUrl': 'https://lda.senate.gov/filings/public/filing/cad6db2f-c3ca-4b9d-bc24-4c56fc7eaadb/print/', 'income': None, 'expenses': 1450000, 'postedName': '', 'date': '', 'clientId': '103979', 'registrantId': '4152', 'senateId': '4152-103979', 'houseRegistrantId': '31450'}, {'symbol': 'AAPL', 'name': 'APPLE INC', 'description': '', 'country': 'US', 'year': 2021, 'period': 'Q1', 'documentUrl': 'https://lda.senate.gov/filings/public/filing/931b23da-8021-4f33-8f4b-70e571dba9b8/print/', 'income': None, 'expenses': 1460000, 'postedName': '', 'date': '', 'clientId': '103979', 'registrantId': '4152', 'senateId': '4152-103979', 'houseRegistrantId': '31450'}, {'symbol': 'AAPL', 'name': 'APPLE INC', 'description': '', 'country': 'US', 'year': 2021, 'period': 'Q2', 'documentUrl': 'https://lda.senate.gov/filings/public/filing/008a3286-d6d5-4601-afe3-f4c4361785fc/print/', 'income':